In [1]:
#Install dependencies
!pip install -q transformers datasets accelerate sentencepiece huggingface_hub


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# -------------------------------
# 1. Imports
# -------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

/usr/local/lib/python3.12/site-packages/torch_xla/__init__.py:258: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.12/site-packages/jax/_src/cloud_tpu_init.py:93: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [3]:
# -------------------------------
# 2. Load Alpaca dataset
# -------------------------------
# Alpaca dataset is hosted on Hugging Face Hub
dataset = load_dataset("tatsu-lab/alpaca")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [4]:
# Inspect one sample
print(dataset["train"][0])

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


In [6]:
# -------------------------------
# 3. Preprocess data
# -------------------------------
model_id = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Fix: set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

def format_example(example):
    # Combine instruction + input into a single prompt
    if example.get("input"):
        prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}\nOutput:"
    else:
        prompt = f"Instruction: {example['instruction']}\nOutput:"
    return {"text": prompt + example["output"]}

formatted_dataset = dataset["train"].map(format_example)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [7]:
# -------------------------------
# 4. Load base model
# -------------------------------
model = AutoModelForCausalLM.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [14]:
!pip install -U transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 45.8 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [17]:
# -------------------------------
# 5. Training setup
# -------------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    optim="adamw_torch_xla"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(1000)),  # small eval subset
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_12/3718424342.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

# -------------------------------
# 6. Train
# -------------------------------
trainer.train()


Step,Training Loss


In [ ]:
#-------------------------------
# 7. Save model
# -------------------------------
trainer.save_model("./alpaca-gptneo-125m")
tokenizer.save_pretrained("./alpaca-gptneo-125m")

# -------------------------------
# 8. Test inference
# -------------------------------
from transformers import pipeline

pipe = pipeline("text-generation", model="./alpaca-gptneo-125m", tokenizer=tokenizer)

print(pipe("Instruction: Write a poem about F1 racing\nOutput:", max_new_tokens=100)[0]["generated_text"])